In [2]:
import numpy as np
from scipy.integrate import quad

# helper funcs for likeli

In [3]:
from scipy.special import erf
def Phi(x):
    """
    Define the normal cumulative distribution function Φ(x) using erf
    """
    return 0.5 * (1 + erf(x / np.sqrt(2)))

def rho_A_t_fn(t, V_A, theta_A):
    """
    For AI,prob density of t given V_A, theta_A
    """
    if t <= 0:
        return 0
    return (theta_A*1/np.sqrt(2*np.pi*(t)**3))*np.exp(-0.5 * (V_A**2) * (((t) - (theta_A/V_A))**2)/(t))


def cum_A_t_fn(t, V_A, theta_A):
    """
    For AI, calculate cummulative distrn of a time t given V_A, theta_A
    """
    if t <= 0:
        return 0

    term1 = Phi(V_A * ((t) - (theta_A/V_A)) / np.sqrt(t))
    term2 = np.exp(2 * V_A * theta_A) * Phi(-V_A * ((t) + (theta_A / V_A)) / np.sqrt(t))
    
    return term1 + term2

# params

In [5]:
V_A = 1.1
theta_A = 1.8
t_stim_0 = 0.2
t_stim_tau = 0.4

# raga

In [22]:
def non_aborts_pdf(t, t_stim_0, t_stim_tau, V_A, theta_A):
    prob_that_stim_occurs_after_t = np.exp(- ( (t - t_stim_0) / t_stim_tau ))
    censor_prob_after_t = (1 - cum_A_t_fn(t, V_A, theta_A))
    return prob_that_stim_occurs_after_t * censor_prob_after_t

def raga_likelihood(t, t_stim_0, t_stim_tau, V_A, theta_A):
    # rt < t_stim
    prob_of_hitting_bound = rho_A_t_fn(t, V_A, theta_A)
    if t < t_stim_0:
        return prob_of_hitting_bound
    else:
        # rt  < t_stim
        prob_of_fix_survive = np.exp(- ( (t - t_stim_0) / t_stim_tau ))
        prob_of_abort = prob_of_fix_survive * prob_of_hitting_bound

        # # rt > t_stim
        # censored_prob = quad(non_aborts_pdf, t_stim_0, 20, args=(t_stim_0, t_stim_tau, V_A, theta_A))[0]  
        # print(f'censored_prob: {censored_prob}')    
        # censored_prob = 1 - cum_A_t_fn(t, V_A, theta_A)      


        return prob_of_abort





In [35]:
area_raga = quad(raga_likelihood, 0, np.inf, args=(t_stim_0, t_stim_tau, V_A, theta_A))[0]
print(area_raga)

0.13456382632379474


In [47]:
def after_abort_fn(t, t_stim_0, t_stim_tau, V_A, theta_A):
    # prob_of_non_abort = 1 - cum_A_t_fn(t, V_A, theta_A)
    # return prob_of_non_abort
    fixation_didnt_survive = 1 - np.exp(- ( (t - t_stim_0) / t_stim_tau ))

    return fixation_didnt_survive*rho_A_t_fn(t, V_A, theta_A)

In [48]:
quad(after_abort_fn, 0, 10, args=(t_stim_0, t_stim_tau, V_A, theta_A))[0]

0.8649626896534214